<a href="https://colab.research.google.com/github/muhammadalvarokhikman/ASL-Education/blob/main/American_Sign_Language_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Flask==2.2.5
!pip install tensorflow==2.9.1
!pip install numpy==1.23.5
!pip install pandas>=1.1.5
!pip install matplotlib>=3.3.4
!pip install scikit-learn>=0.24.2
!pip install Pillow>=8.4.0
!pip install h5py>=3.1.0

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Parameter
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Fungsi untuk memproses gambar
def process_image(file_path, label):
    # Load gambar
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = img / 255.0  # Normalisasi ke [0, 1]
    return img, label

# Fungsi untuk membuat dataset training
def create_train_dataset(directory):
    classes = sorted(os.listdir(directory))  # Subfolder sebagai kelas
    class_indices = {cls: i for i, cls in enumerate(classes)}  # Mapping kelas ke indeks

    file_paths = []
    labels = []

    # Loop melalui folder kelas
    for cls in classes:
        cls_dir = os.path.join(directory, cls)
        for file_name in os.listdir(cls_dir):
            file_paths.append(os.path.join(cls_dir, file_name))
            labels.append(class_indices[cls])

    # Buat dataset TensorFlow
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset, classes

# Fungsi untuk membuat dataset test
def create_test_dataset(directory, classes):
    file_paths = []
    labels = []

    # Mapping kelas ke indeks berdasarkan classes dari train dataset
    class_indices = {cls: i for i, cls in enumerate(classes)}

    # Loop melalui semua file di dalam folder test
    for file_name in os.listdir(directory):
        if file_name.endswith(".jpg"):  # Pastikan hanya file gambar
            class_name = file_name.split("_test")[0]  # Ambil huruf dari nama file
            file_paths.append(os.path.join(directory, file_name))
            labels.append(class_indices[class_name])

    # Buat dataset TensorFlow
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    return dataset

In [ ]:
# Path dataset
train_dir = "/content/drive/MyDrive/American Sign Languages/asl_alphabet_train"
test_dir = "/content/drive/MyDrive/American Sign Languages/asl_alphabet_test"

# Buat dataset
train_dataset, classes = create_train_dataset(train_dir)
test_dataset = create_test_dataset(test_dir, classes)

In [ ]:
# Load MobileNetV2
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights="imagenet")
base_model.trainable = False

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
# Tambahkan lapisan custom untuk klasifikasi
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.2),
    Dense(len(classes), activation="softmax")  # Output sesuai jumlah kelas
])

In [ ]:
# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Checkpoint untuk menyimpan model terbaik
checkpoint = ModelCheckpoint("asl_model.keras", monitor="val_accuracy", save_best_only=True, mode="max")

# Latih model
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCHS,
    callbacks=[checkpoint]
)

Epoch 1/10
91/91 [==============================] - 170s 2s/step - loss: 2.4714 - accuracy: 0.4321 - val_loss: 1.5554 - val_accuracy: 0.4643
Epoch 2/10
91/91 [==============================] - 158s 2s/step - loss: 0.5811 - accuracy: 0.8821 - val_loss: 0.8127 - val_accuracy: 0.8214
Epoch 3/10
91/91 [==============================] - 156s 2s/step - loss: 0.2563 - accuracy: 0.9614 - val_loss: 0.5030 - val_accuracy: 0.8929
Epoch 4/10
91/91 [==============================] - 159s 2s/step - loss: 0.1525 - accuracy: 0.9834 - val_loss: 0.3232 - val_accuracy: 0.9643
Epoch 5/10
91/91 [==============================] - 160s 2s/step - loss: 0.1042 - accuracy: 0.9917 - val_loss: 0.2571 - val_accuracy: 1.0000
Epoch 6/10
91/91 [==============================] - 154s 2s/step - loss: 0.0782 - accuracy: 0.9931 - val_loss: 0.1815 - val_accuracy: 1.0000
Epoch 7/10
91/91 [==============================] - 154s 2s/step - loss: 0.0553 - accuracy: 0.9976 - val_loss: 0.1437 - val_accuracy: 1.0000
Epoch 8/10
91

In [ ]:
# Simpan model akhir
model.save("final_asl_model.keras")

# Simpan model akhir dalam format .h5
model.save("final_asl_model.h5")

In [ ]:
# Muat model .h5
model_h5 = tf.keras.models.load_model("final_asl_model.h5")

# Muat model .keras
model_keras = tf.keras.models.load_model("final_asl_model.keras")